# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

# Hackathon : Voice commands based food ordering system

### Setup Steps

In [0]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "P181902276" #@param {type:"string"}


In [0]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "9848253595" #@param {type:"string"}


In [0]:
#@title Run this cell to complete the setup for this Notebook

from IPython import get_ipython
ipython = get_ipython()
  
notebook="BLR_M2_Hackathon" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/Noisy_data.zip")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/studio_data.zip")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/Record_audio.py")
    ipython.magic("sx wget https://cdn.talentsprint.com/aiml/Experiment_related_data/Week8/Hackathon2/net_speech_89.pt")
    ipython.magic("sx unzip studio_data.zip")
    ipython.magic("sx unzip Noisy_data.zip")
    ipython.magic("sx pip install torch torchvision")
    ipython.magic("sx pip install librosa")
    print ("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "id" : Id, "file_hash" : file_hash, "notebook" : notebook}

      r = requests.post(url, data = data)
      print("Your submission is successful. Ref:", submission_id)
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if Additional: return Additional      
    else: raise NameError('')
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
  
else:
  print ("Please complete Id and Password cells before running setup")



In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import librosa
import os
import warnings
from time import sleep
import sys
warnings.filterwarnings('ignore')

## Pretrained Network for deep features


The following function contains code to load a pre-trained network to produces deep features for the audio sample. This network is trained with delta MFCC features of mono channel 8000 bit rate audio sample.

In [0]:
def get_network():

    net = torch.nn.Sequential()

    saved_net = torch.load("net_speech_89.pt").cpu()

    for index, module in enumerate(saved_net):
        net.add_module("layer"+str(index),module)
        if (index+1)%17 == 0 :
            break
    return net

In [0]:
get_network()

Sequential(
  (layer0): Linear(in_features=900, out_features=800, bias=True)
  (layer1): ReLU()
  (layer2): Linear(in_features=800, out_features=700, bias=True)
  (layer3): ReLU()
  (layer4): Linear(in_features=700, out_features=600, bias=True)
  (layer5): ReLU()
  (layer6): Linear(in_features=600, out_features=500, bias=True)
  (layer7): ReLU()
  (layer8): Linear(in_features=500, out_features=400, bias=True)
  (layer9): ReLU()
  (layer10): Linear(in_features=400, out_features=300, bias=True)
  (layer11): ReLU()
  (layer12): Linear(in_features=300, out_features=200, bias=True)
  (layer13): ReLU()
  (layer14): Linear(in_features=200, out_features=100, bias=True)
  (layer15): ReLU()
  (layer16): Linear(in_features=100, out_features=50, bias=True)
)

##Obtaining Features from Audio samples
Generate features from a audio sample of '.wav' format
* Generate Delta MFCC features of order 1 and 2 
* Passes them through the above mentioned deep neural net
* the obtained deep features are returned

Parameters: Filepath (path of audio sample),
                       sr (sampling rate, all the samples provided are of 8000 bitrate)
         
  Caution: Do not change the default parameters

In [0]:
def get_features(filepath, sr=8000, n_mfcc=30, n_mels=128, frames = 15):
    
    
    y, sr = librosa.load(filepath, sr=sr)
    D = np.abs(librosa.stft(y))**2
    S = librosa.feature.melspectrogram(S=D)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    log_S = librosa.power_to_db(S,ref=np.max)
    features = librosa.feature.mfcc(S=log_S, n_mfcc=n_mfcc)
    if features.shape[1] < frames :
        features = np.hstack((features, np.zeros((n_mfcc, frames - features.shape[1]))))
    elif features.shape[1] > frames:
        features = features[:, :frames]
    # Find 1st order delta_mfcc
    delta1_mfcc = librosa.feature.delta(features, order=1)

    # Find 2nd order delta_mfcc
    delta2_mfcc = librosa.feature.delta(features, order=2)
    features = np.hstack((delta1_mfcc.flatten(), delta2_mfcc.flatten()))
    features = features.flatten()[np.newaxis, :]
    features = Variable(torch.from_numpy(features)).float()
    deep_net = get_network()
    deep_features = deep_net(features)
    #print(features.shape)
    #print(audio_file)
    #features.flatten()[np.newaxis, :]
    return deep_features.data.numpy().flatten()

## All the voice sample needed for training are present across the folders "Noisy_data" and "studio_data"

In [0]:
%ls

BLR_M2_Hackathon.ipynb  Noisy_data/      sample_data/
__MACOSX/               Noisy_data.zip   studio_data/
net_speech_89.pt        Record_audio.py  studio_data.zip


##Stage 1: 
#### Max. Marks: 5
#### Complete the code in the load_data function

* The function should take path of the folder containing audio samples as input
* It should return features of all the audio samples present in the specified folder into single array (list of lists or 2-d numpy array) and their respective labels should be returned too

In [1]:
import os
import numpy as np
import re

def load_data(path):
    files = []
    feat=[]
    label=[]
    for r,d,f in os.walk(path):
        for file in f:
            if '.wav' in file:
                files.append(os.path.join(r,file))
    for f in files:
        feat.append(get_features(f))
        label.append(re.search(r'\d+',f).group(0))
   
    features=np.asarray(feat)
    labels=np.asarray(label)
    return features, labels


In [2]:
load_data('studio_data')

(array([], dtype=float64), array([], dtype=float64))

####load data and labels from studio_data folder

In [3]:
studio_recorded_features, studio_recorded_labels = load_data('/content/studio_data/')

In [4]:
print(studio_recorded_features.shape,studio_recorded_labels.shape)

(0,) (0,)


In [5]:
len(studio_recorded_labels)

0

##Stage 2:

####Max. Marks:18

#### Train a classifier on the features obtained above

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
values_of_k = [1,3,5,7,9,11,13,15,17,19]

for i in range(len(values_of_k)):
    knc = KNeighborsClassifier(values_of_k[i])
    X_train,X_test,y_train,y_test = train_test_split(studio_recorded_features,studio_recorded_labels,test_size=0.1)
    knc.fit(X_train,y_train)
    # print(X_train.shape,y_train.shape)
    predict =knc.predict(X_test)
    acuracy = accuracy_score(y_test,predict)
    print('Values of K: ',values_of_k[i])
    acuracy
    print('Accuracy: ',acuracy)
    



Values of K:  1
Accuracy:  0.8766310794780545
Values of K:  3
Accuracy:  0.863582443653618
Values of K:  5
Accuracy:  0.8754448398576512
Values of K:  7
Accuracy:  0.9098457888493475
Values of K:  9
Accuracy:  0.8612099644128114
Values of K:  11
Accuracy:  0.8884934756820878
Values of K:  13
Accuracy:  0.8979833926453143
Values of K:  15
Accuracy:  0.8861209964412812
Values of K:  17
Accuracy:  0.8718861209964412
Values of K:  19
Accuracy:  0.8956109134045077


In [0]:
print(knc.predict_proba)

<bound method KNeighborsClassifier.predict_proba of KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=19, p=2,
                     weights='uniform')>


####Save your model

Hint:
* Incase if you are using scikit learn model for training, you can use joblib package to save the model.
* Manually implemented models as a function or class can be saved using pickle 

In [0]:
from joblib import dump, load

In [0]:
# from sklearn.externals import joblib
filename = 'finalized_model_std.sav'
joblib.dump(knc, filename)

In [0]:
dump(knc, 'finalized_model_std.sav')

['finalized_model_std.sav']

In [0]:
  !pip freeze | grep joblib

joblib==0.12.5


#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download('finalized_model_std.sav')

In [0]:
%ls


##Stage 3: (Application Deployment)

#### Max. Marks: 5M
#### Now deploy the model trained on studio_data in the sever to order food correctly. 
#### Deployment instruction are given the Hackathon documentation
#### After deploying and checking the application come back here to train on Noisy_data to generalise better in real situations

#### load data and labels from Noisy_data folder

In [0]:
noisy_data, noisy_data_labels = load_data('/content/Noisy_data')

In [0]:
print(studio_recorded_features.shape,noisy_data.shape)
merg_feature = np.concatenate((studio_recorded_features, noisy_data))
print(merg_feature.shape)
# studio_recorded_features, studio_recorded_labels

(8423, 50) (11839, 50)
(20262, 50)


In [0]:
merg_labels = np.concatenate((studio_recorded_labels, noisy_data_labels))

In [0]:
merg_labels.shape

(20262,)

In [0]:
studio_recorded_labels.shape

(8423,)

## Stage 4:
####Max. Marks:8
#### Train a classifier on the features obtained from both the Noisy_data and Studio_data

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
values_of_k = [1,3,5,7,9,11]

for i in range(len(values_of_k)):
    mknc = KNeighborsClassifier(values_of_k[i])
    X_train,X_test,y_train,y_test = train_test_split(merg_feature,merg_labels,test_size=0.1)
    mknc.fit(X_train,y_train)
    # print(X_train.shape,y_train.shape)
    mpredict =mknc.predict(X_test)
    macuracy = accuracy_score(y_test,mpredict)
    print('Value of k: ',values_of_k[i])
    print('Accuracy: ',macuracy)

Value of k:  1
Accuracy:  0.4814997533300444
Value of k:  3
Accuracy:  0.5051800690675876
Value of k:  5
Accuracy:  0.5515540207202763
Value of k:  7
Accuracy:  0.5624074987666502
Value of k:  9
Accuracy:  0.5609274790330537
Value of k:  11
Accuracy:  0.5693142575234337


#### Download your trained model using the code below
* given the path of model file the following code downloads it through the browser

In [0]:
from google.colab import files
files.download('<model_file_path>')

In [0]:
from joblib import dump, load

In [0]:
dump(mknc, 'merged_model_std.sav')

['merged_model_std.sav']

#### Now deploy the model trained above in the sever to order food correctly. 
#### Deployment instruction are given the Hackathon documentation
#### After deploying and checking the application, record your teams data from the web application provided in the Hackathon document

In [0]:
!mkdir teamdata

Replace <YOUR_GROUP_ID> with your group id given in the lab

In [0]:
!wget -r -A .wav https://aiml-sandbox1.talentsprint.com/audio_recorder/b9h1g16/team_data/ -nH --cut-dirs=100  -P ./team_data

In [0]:
%ls

BLR_M2_Hackathon.ipynb   net_speech_89.pt  sample_data/     team_data/
finalized_model_std.pkl  Noisy_data/       studio_data/
finalized_model_std.sav  Noisy_data.zip    studio_data.zip
__MACOSX/                Record_audio.py   teamdata/


In [0]:
!unzip <zip_file_name>

##Stage 5:
#### Max. Marks: 14

## Enhance the model trained with both the noisy data and studio_data to your team's voice samples

In [0]:
team_recorded_features, team_recorded_labels = load_data('/content/team_data/')

In [0]:
print(team_recorded_features.shape)

(60, 50)


In [0]:
team_features = np.concatenate((team_recorded_features, merg_feature))
print(team_features.shape)
# team_labels = np.concatenate(team_recorded_labels,merg_labels)

(20322, 50)


In [0]:
team_labels = np.concatenate((team_recorded_labels,merg_labels))
print(team_labels.shape)

(20322,)


In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
values_of_k = [1,3,5,7,9,11,13,15,17,19]

for i in range(len(values_of_k)):
    tknc = KNeighborsClassifier(values_of_k[i])
    X_train,X_test,y_train,y_test = train_test_split(team_features,team_labels,test_size=0.1)
    tknc.fit(X_train,y_train)
    # print(X_train.shape,y_train.shape)
    tpredict =tknc.predict(X_test)
    tacuracy = accuracy_score(y_test,tpredict)
    tacuracy
    print('Value of K: ',values_of_k[i])
    print('Accuracy: ',tacuracy)

Value of K:  1
Accuracy:  0.485489424495819
Value of K:  3
Accuracy:  0.5218888342351206
Value of K:  5
Accuracy:  0.52926709296606
Value of K:  7
Accuracy:  0.5568125922282341
Value of K:  9
Accuracy:  0.5646827348745695
Value of K:  11
Accuracy:  0.5558288243974422
Value of K:  13
Accuracy:  0.5602557796360059
Value of K:  15
Accuracy:  0.5553369404820462
Value of K:  17
Accuracy:  0.5479586817511067
Value of K:  19
Accuracy:  0.5538612887358584


In [0]:
dump(tknc, 'team_model_std.sav')

['team_model_std.sav']

### Now deploy the model trained above in the sever to order food correctly. 
### Deployment instruction are given the Hackathon documentation

### Please answer the questions below to complete the experiment:

In [0]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging me" #@param ["Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging me", "Was Tough, but I did it", "Too Difficult for me"]


In [0]:
#@title If it was very easy, what more you would have liked to have been added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "NA" #@param {type:"string"}

In [0]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["Yes", "No"]4819687275

In [0]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")